In [1]:
import tensorflow as tf
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import random_seed
from tensorflow.contrib.learn.python.learn.datasets import base
import math
import numpy as np
import pandas as pd
import functools as ft
import csv
import os
import random


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_csv(fname, col_start=1, row_start=1, delimiter=",", dtype=dtypes.float32):
    data = np.genfromtxt(fname, delimiter=delimiter)
    for _ in range(col_start):
        data = np.delete(data, (0), axis=1)
    for _ in range(row_start):
        data = np.delete(data, (0), axis=0)
    # remove two unnecessary columns
    data = data[:,:5]
    # use less data for test
    data = data[2000000:2010000]
    print('data.shape ', data.shape)
    # print(np.transpose(data))
    return data



In [4]:
def process_data(data):
    columns = 5
    moving_window = 128
    stock_set = np.zeros([0,2,moving_window,columns])
    label_set = np.zeros([0,2])
    rising_idx = 0
    fall_idx = 0
    # start from 1152 because we need to add 128 prices for each 10 minutes in the past
    for idx in range(1152,data.shape[0] - (moving_window + 5)):
#       min_data     = data[range(idx,idx+(moving_window)),:]
#       e10_min_data = data[range((idx+moving_window)-(moving_window*10),idx+(moving_window),10)]
#       temp = []
#       temp.append(min_data)
#       temp.append(e10_min_data)
        if idx % 500 == 0:
            print("index: " , idx)
          # OLD if data[idx+(moving_window+5),3] > data[idx+(moving_window),3]:
          # true if the price will rise at least 0.1 %
        average_price_in_5_min = np.average(data[range((idx + (moving_window)),(idx + (moving_window + 5))),3])
        if average_price_in_5_min > (data[idx + (moving_window), 3]*1.00001):
            rising_idx +=1
            lbl = [[1.0, 0.0]]
            stock_set = np.concatenate((stock_set, np.expand_dims([data[range(idx,idx+(moving_window)),:],data[range((idx+moving_window)-(moving_window*10),idx+(moving_window),10)]], axis=0)), axis=0)
            label_set = np.concatenate((label_set, lbl), axis=0)
            #print("Price has risen!!!")
        else:
            fall_idx +=1
            lbl = [[0.0, 1.0]]
            # 33% chance to not use this window.
            if(random.choice([0,1])==0):
                stock_set = np.concatenate((stock_set, np.expand_dims([data[range(idx,idx+(moving_window)),:],data[range((idx+moving_window)-(moving_window*10),idx+(moving_window),10)]], axis=0)), axis=0)
                label_set = np.concatenate((label_set, lbl), axis=0)
       # if idx % 500 == 0:
            #print('label_set added %s average price: %f , last: %f'  % (lbl,average_price_in_5_min,data[idx + (moving_window), 3]*1.0001 ))  
      # label_set = np.concatenate((label_set, np.array([data[idx+(moving_window+5),3] - data[idx+(moving_window),3]])))
        if idx % 5000 == 0:
            index = idx/5000
            np.save('data/stock_set_'+str(index),stock_set)
            np.save('data/label_set_'+str(index),label_set)
            print("Saved!")
            stock_set = np.zeros([0,2,moving_window,columns])
            label_set = np.zeros([0,2])  
    print(stock_set.shape, label_set.shape)
   # return stock_set, label_set

In [5]:
process_data(load_csv("..//input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2018-01-08.csv"))

data.shape  (10000, 5)
index:  1500
index:  2000
index:  2500
index:  3000
index:  3500
index:  4000
index:  4500
index:  5000
Saved!
index:  5500
index:  6000
index:  6500
index:  7000
index:  7500
index:  8000
index:  8500
index:  9000
index:  9500
(3650, 2, 128, 5) (3650, 2)


1